In [2]:
import pandas as pd
import os
import csv

# Build the CYC (incorrectly labeled CPR hereafter)

In [309]:
def build_cpr(a:bool, b:bool, c:bool, d:bool):
    """
    Build the constrictive presence ratio CSV in every play's tab
    a: weighted_blockers
    b: weighted_no_blockers
    c: unweighted_blockers
    d: unweighted_no_blockers
    """    
    # Read the 'eval_frame_df.csv' file
    eval_frame_df = pd.read_csv(f'./data/eval_frame_df_old.csv')
    finished = set()

    # Iterate through rows of 'eval_frame_df'
    for _, row in eval_frame_df[['gameId', 'playId']].drop_duplicates().iterrows():
        gameId = int(row['gameId'])
        playId = int(row['playId'])

        # if (gameId != 2022101610) | (playId != 3730):
        #     continue

        if (gameId, playId) in finished: 
            continue
        
        # Look up the game in 'games.csv'
        games_df = pd.read_csv('./data/games.csv')
        game_info = games_df[(games_df['gameId'] == gameId)]

        if game_info.empty:
            print(f"No game found for gameId: {gameId}")
            continue

        homeTeamAbbr = game_info['homeTeamAbbr'].values[0]
        visitingTeamAbbr = game_info['visitorTeamAbbr'].values[0]

        # Define the folder path
        folder_path = f'./games/{gameId}_{homeTeamAbbr}_{visitingTeamAbbr}/{playId}/'

        # Check if the folder exists and if the file exists
        if not os.path.exists(folder_path):
            print(f"Folder not found for gameId: {folder_path}")
            finished.add((gameId, playId))
            continue

        print(gameId, playId)

        if a: 
            weighted_blockers = os.path.join(folder_path, 'tpc_per_frame_weighted_blockers.csv')
        if b:
            weighted_no_blockers = os.path.join(folder_path, 'tpc_per_frame_weighted_no_blockers.csv')
        if c: 
            unweighted_no_blockers = os.path.join(folder_path, 'tpc_per_frame_unweighted_no_blockers.csv')
        if d: 
            unweighted_blockers = os.path.join(folder_path, 'tpc_per_frame_unweighted.csv')

        # if not os.path.exists(weighted_no_blockers):
        if not (os.path.exists(weighted_blockers) and os.path.exists(weighted_no_blockers)):
            print(f"File not found for gameId: {gameId}, {playId}")
            finished.add((gameId, playId))
            continue

        # Read 'tpc_per_frame_weighted_blockers.csv'
        if a: 
            try:
                tpc_per_frame_weighted_blockers = pd.read_csv(weighted_blockers, index_col=0)
                tpc_per_frame_weighted_blockers = tpc_per_frame_weighted_blockers.fillna(0)
                # if tpc_per_frame_weighted_blockers.isna().any().any():
                #     print('found nan')
                #     continue
            except:
                continue
        if b: 
            try:
                tpc_per_frame_weighted_no_blockers = pd.read_csv(weighted_no_blockers, index_col=0)
                tpc_per_frame_weighted_no_blockers = tpc_per_frame_weighted_no_blockers.fillna(0)
                # if tpc_per_frame_weighted_no_blockers.isna().any().any():
                #     print('found nan')
                #     continue
            except:
                continue
        if c: 
            tpc_per_frame_unweighted_blockers = pd.read_csv(unweighted_blockers, index_col=0)
            tpc_per_frame_unweighted_blockers = tpc_per_frame_unweighted_blockers.fillna(0)
        if d: 
            tpc_per_frame_unweighted_no_blockers = pd.read_csv(unweighted_no_blockers, index_col=0)
            tpc_per_frame_unweighted_no_blockers = tpc_per_frame_unweighted_no_blockers.fillna(0)

        # Sort 'eval_frame_df' by matching gameId and playId and sort it in order of frameId
        sorted_eval_frame_df = eval_frame_df[(eval_frame_df['gameId'] == gameId) & (eval_frame_df['playId'] == playId)].sort_values(by='frameId')
        filtered_eval_frame_df = sorted_eval_frame_df[sorted_eval_frame_df['frameId'].isin(tpc_per_frame_weighted_no_blockers.index)]
        ######### DEBUG #########################
        # print(filtered_eval_frame_df.index)


        # Calculate the product
        expected_yards_diff = filtered_eval_frame_df['expectedYardsByCarrier'].diff(-1).fillna(0)
        expected_yards_diff.index = tpc_per_frame_weighted_blockers.index
        
        # print(tpc_per_frame_weighted_blockers)

        # print("------------")

        # print(expected_yards_diff)

        # print("------------")

        # Check if the length of expected_yards_diff is the same as the number of rows in tpc_per_frame_weighted_blockers
        if (len(expected_yards_diff) == len(tpc_per_frame_weighted_blockers)) and (len(expected_yards_diff) == len(tpc_per_frame_weighted_no_blockers)):
            # Multiply each entry in the corresponding row by the value in expected_yards_diff
            if a: 
                cpr_weighted_blockers = tpc_per_frame_weighted_blockers.multiply(expected_yards_diff, axis=0)
            if b: 
                cpr_weighted_no_blockers = tpc_per_frame_weighted_no_blockers.multiply(expected_yards_diff, axis=0)
            if c: 
                cpr_unweighted_blockers = tpc_per_frame_unweighted_blockers.multiply(expected_yards_diff, axis=0)
            if d: 
                cpr_unweighted_no_blockers = tpc_per_frame_unweighted_no_blockers.multiply(expected_yards_diff, axis=0)
        else:
            # Print the shapes of both dataframes
            print(f"Shapes do not match - expected_yards_diff shape: {len(expected_yards_diff)}, tpc_per_frame_weighted_blockers shape: {len(tpc_per_frame_weighted_blockers)}")
            finished.add((gameId, playId))

        if a:
            cpr_weighted_blockers.to_csv(os.path.join(folder_path, 'cpr_weighted_blockers.csv'))
        if b: 
            cpr_weighted_no_blockers.to_csv(os.path.join(folder_path, 'cpr_weighted_no_blockers.csv'))
        if c: 
            cpr_unweighted_blockers.to_csv(os.path.join(folder_path, 'cpr_unweighted_blockers.csv'))
        if d: 
            cpr_unweighted_no_blockers.to_csv(os.path.join(folder_path, 'cpr_unweighted_no_blockers.csv'))

        # print(cpr_weighted_blockers.sum())

        # print("------------")

        finished.add((gameId, playId))

        print(f'success for {gameId}, {playId}')


In [7]:
def build_cpr_optimized(a: bool, b: bool, c: bool, d: bool):
    eval_frame_df = pd.read_csv('./data/eval_frame_df_old.csv')
    games_df = pd.read_csv('./data/games.csv')
    finished = set()

    for (gameId, playId), group in eval_frame_df.groupby(['gameId', 'playId']):
        if (gameId, playId) in finished:
            continue

        game_info = games_df[games_df['gameId'] == gameId]
        if game_info.empty:
            print(f"No game found for gameId: {gameId}")
            continue

        homeTeamAbbr, visitingTeamAbbr = game_info.iloc[0][['homeTeamAbbr', 'visitorTeamAbbr']]
        folder_path = f'./games/{gameId}_{homeTeamAbbr}_{visitingTeamAbbr}/{playId}/'

        if not os.path.exists(folder_path):
            print(f"Folder not found for gameId: {folder_path}")
            finished.add((gameId, playId))
            continue

        files_to_check = [os.path.join(folder_path, f'tpc_per_frame_{"weighted" if flag else "unweighted"}_{"blockers" if flag else "no_blockers"}.csv') 
                          for flag in (a, b, c, d)]
        if not all(os.path.exists(file) for file in files_to_check):
            print(f"File not found for gameId: {gameId}, {playId}")
            finished.add((gameId, playId))
            continue

        sorted_eval_frame_df = group.sort_values(by='frameId')

        for flag, file_suffix in zip((a, b, c, d), ['weighted_blockers', 'weighted_no_blockers', 'unweighted_blockers', 'unweighted_no_blockers']):
            if flag:
                file_path = os.path.join(folder_path, f'tpc_per_frame_{file_suffix}.csv')
                try:
                    tpc_data = pd.read_csv(file_path, index_col=0).fillna(0)
                    filtered_df = sorted_eval_frame_df[sorted_eval_frame_df['frameId'].isin(tpc_data.index)]
                    expected_yards_diff = filtered_df['expectedYardsByCarrier'].diff(-1).fillna(0)
                    cpr_data = tpc_data.multiply(expected_yards_diff, axis=0)
                    cpr_data.to_csv(os.path.join(folder_path, f'cpr_{file_suffix}.csv'))
                except Exception as e:
                    print(f"Error processing file: {file_path} - {e}")
                    continue

        finished.add((gameId, playId))
        print(f'Success for {gameId}, {playId}')


In [8]:
def stitch_cpr_optimized(output_filename:str='cpr_weighted_blockers'): 
    games_folder = './games'
    weighted_blockers_output_file = f'./data/{output_filename}.csv'
    eval_frame_df = pd.read_csv('./data/eval_frame_df.csv')

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(weighted_blockers_output_file), exist_ok=True)

    # Pre-read 'games.csv' outside the loop
    games_df = pd.read_csv('./data/games.csv')

    # Open the CSV file for writing
    with open(weighted_blockers_output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['gameId', 'playId', 'nflId', 'constrictivePresenceSum'])

        # Iterate over unique (gameId, playId) pairs
        for (gameId, playId), group in eval_frame_df.groupby(['gameId', 'playId']):
            game_info = games_df[games_df['gameId'] == gameId]
            if game_info.empty:
                print(f"No game found for gameId: {gameId}")
                continue

            homeTeamAbbr = game_info.iloc[0]['homeTeamAbbr']
            visitingTeamAbbr = game_info.iloc[0]['visitorTeamAbbr']
            folder_path = f'./games/{gameId}_{homeTeamAbbr}_{visitingTeamAbbr}/{playId}/'

            weighted_blockers_filepath = os.path.join(folder_path, 'cpr_weighted_blockers.csv')
            if not os.path.exists(weighted_blockers_filepath):
                print(f"No file found at {folder_path}")
                continue

            try:
                weighted_blockers_df = pd.read_csv(weighted_blockers_filepath, index_col=0)
                player_sums = weighted_blockers_df.sum()

                for playerId, sum_value in player_sums.items():
                    writer.writerow([gameId, playId, int(playerId), sum_value])
            except Exception as e:
                print(f"Error processing file: {weighted_blockers_filepath} - {e}")
            continue

In [ ]:
build_cpr(a=True, b=True, c=False, d=False)

In [311]:
###### aggregate all players into one df
# Define the path to the 'games' folder and the output file
games_folder = './games'
games_df = pd.read_csv('data/games.csv')
weighted_blockers_output_file = './data/cpr_weighted_blockers.csv'
weighted_no_blockers_output_file = './data/cpr_weighted_no_blockers.csv'
eval_frame_df = pd.read_csv(f'./data/eval_frame_df.csv')
# Ensure the output directory exists
os.makedirs(os.path.dirname(weighted_blockers_output_file), exist_ok=True)
os.makedirs(os.path.dirname(weighted_no_blockers_output_file), exist_ok=True)

# Open the CSV file for writing
finished = set()

with open(weighted_blockers_output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['gameId', 'playId', 'nflId', 'constrictivePresenceSum'])

    # iterate over the eval_frame_df
    for _, row in eval_frame_df.iterrows():
        gameId = int(row['gameId'])
        playId = int(row['playId'])
        if (gameId, playId) in finished: 
            continue
        
        # Look up the game in 'games.csv'
        game_info = games_df[(games_df['gameId'] == gameId)]

        if game_info.empty:
            print(f"No game found for gameId: {gameId}")
            continue

        homeTeamAbbr = game_info['homeTeamAbbr'].values[0]
        visitingTeamAbbr = game_info['visitorTeamAbbr'].values[0]

        # Define the folder path
        folder_path = f'./games/{gameId}_{homeTeamAbbr}_{visitingTeamAbbr}/{playId}/'
        try: 
            weighted_blockers_filepath = os.path.join(folder_path, 'cpr_weighted_blockers.csv')
            weighted_blockers_df = pd.read_csv(weighted_blockers_filepath, index_col=0)
        except: 
            print(f"No file found at {folder_path}")
            finished.add((gameId, playId))
            continue

        # Calculate the sum of each column (player)
        player_sums = weighted_blockers_df.sum()

        # Write each player's sum to the CSV file
        for playerId, sum_value in player_sums.items():
            writer.writerow([gameId, playId, int(playerId), sum_value])

    finished.add((gameId, playId))

finished = set()
# Open the CSV file for writing
with open(weighted_no_blockers_output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['gameId', 'playId', 'nflId', 'constrictivePresenceSum'])

    # iterate over the eval_frame_df
    for _, row in eval_frame_df.iterrows():
        gameId = int(row['gameId'])
        playId = int(row['playId'])
        if (gameId, playId) in finished: 
            continue
        
        # Look up the game in 'games.csv'
        game_info = games_df[(games_df['gameId'] == gameId)]

        if game_info.empty:
            print(f"No game found for gameId: {gameId}")
            continue

        homeTeamAbbr = game_info['homeTeamAbbr'].values[0]
        visitingTeamAbbr = game_info['visitorTeamAbbr'].values[0]

        # Define the folder path
        folder_path = f'./games/{gameId}_{homeTeamAbbr}_{visitingTeamAbbr}/{playId}/'

        try: 
            weighted_no_blockers_filepath = os.path.join(folder_path, 'cpr_weighted_no_blockers.csv')
            weighted_no_blockers_df = pd.read_csv(weighted_no_blockers_filepath, index_col=0)
        except: 
            print(f"No file found at {folder_path}")
            finished.add((gameId, playId))
            continue

        # Calculate the sum of each column (player)
        player_sums = weighted_no_blockers_df.sum()

        # Write each player's sum to the CSV file
        for playerId, sum_value in player_sums.items():
            writer.writerow([gameId, playId, int(playerId), sum_value])

    finished.add((gameId, playId))

print("Aggregation complete. Data saved")

Aggregation complete. Data saved


In [312]:
def augment_data_file(filename:str):
# augment with additional data like position and name

    # Load the players DataFrame
    players_file = './data/players.csv'
    players_df = pd.read_csv(players_file)

    # Load the constrictive presence ratio DataFrame
    constrictive_file = f'./data/{filename}'
    constrictive_df = pd.read_csv(constrictive_file)

    # Merge the DataFrames on nflId
    augmented_df = pd.merge(constrictive_df, players_df[['nflId', 'position', 'displayName']], 
                            on='nflId', how='left')

    # Save the augmented DataFrame
    augmented_df.to_csv(f'./data/{filename}', index=False)

    print(f"Augmentation complete. Data saved to './data/{filename}'")

In [313]:
cpr_weighted_blockers_df = pd.read_csv('./data/cpr_weighted_blockers.csv')
cpr_weighted_blockers_df[cpr_weighted_blockers_df['nflId'] == 44813].groupby(['gameId', 'playId']).ngroups

176

In [314]:
augment_data_file('cpr_weighted_blockers.csv')

Augmentation complete. Data saved to './data/cpr_weighted_blockers.csv'


In [315]:
augment_data_file('cpr_weighted_no_blockers.csv')

Augmentation complete. Data saved to './data/cpr_weighted_no_blockers.csv'


In [316]:
def group_by_players(filename:str): 
    """ 
    Group the raw data by the players
    """
    # Load the constrictive_presence_ratio_all DataFrame
    constrictive_presence_ratio_all = pd.read_csv(f'./data/{filename}').drop_duplicates()

    # Group by 'displayName' and sum 'constrictivePresenceSum'
    grouped_by_player = constrictive_presence_ratio_all[['nflId', 'displayName', 'position', 'constrictivePresenceSum']].groupby(['nflId', 'displayName', 'position']).sum()

    # Count the number of occurrences for each 'displayName'
    counts = constrictive_presence_ratio_all[['nflId', 'displayName', 'position']].value_counts()
    std_player = constrictive_presence_ratio_all[['nflId',  'displayName', 'position', 'constrictivePresenceSum']].groupby(['nflId', 'displayName', 'position']).std()

    # Divide the summed 'constrictivePresenceSum' by the count for each player
    grouped_by_player['counts'] = counts # - std_player.constrictivePresenceSum / counts
    grouped_by_player['std'] = std_player
    grouped_by_player['cpy_per_play'] = grouped_by_player.constrictivePresenceSum / grouped_by_player.counts

    grouped_by_player.to_csv(f'./data/per_player_{filename}', index=True)

In [317]:
group_by_players('cpr_weighted_blockers.csv')

In [318]:
group_by_players('cpr_weighted_no_blockers.csv')

In [319]:
cpr_weighted_no_blockers = pd.read_csv('./data/per_player_cpr_weighted_no_blockers.csv')
cpr_weighted_no_blockers[cpr_weighted_no_blockers.counts > 150].sort_values('cpy_per_play', ascending=False).head(20)

,nflId,displayName,position,constrictivePresenceSum,counts,std,cpy_per_play
71,41341,DaQuan Jones,DT,39.492731,158,1.294679,0.249954
623,53487,Nick Bolton,ILB,57.619588,258,1.283693,0.223332
248,44976,Matt Milano,OLB,31.281783,194,1.146820,0.161246
15,37104,Cameron Heyward,DT,29.552299,189,0.978925,0.156361
61,41239,Aaron Donald,DT,32.946908,231,0.943170,0.142627
602,53445,Zaven Collins,OLB,40.029766,302,0.939973,0.132549
624,53489,Pete Werner,OLB,30.766078,248,0.834888,0.124057
809,54936,Kader Kohou,CB,24.369838,211,1.427809,0.115497
510,52464,Raekwon Davis,NT,19.049253,179,0.754971,0.106420
417,47877,Jamel Dean,CB,32.230377,315,0.927555,0.102319


In [29]:
cpr_weighted_blockers = pd.read_csv('./data/per_player_cpr_weighted_blockers.csv')
cpr_weighted_blockers[cpr_weighted_blockers.counts > 250].sort_values('cpy_per_play', ascending=False).shape(20)

,nflId,displayName,position,constrictivePresenceSum,counts,std,cpy_per_play
61,41239,Aaron Donald,DT,32.710254,231,0.994126,0.141603
388,47802,Jeffery Simmons,DT,21.464210,205,0.600451,0.104703
623,53487,Nick Bolton,ILB,24.799451,258,1.152403,0.096122
417,47877,Jamel Dean,CB,29.080670,315,1.172624,0.092320
624,53489,Pete Werner,OLB,21.379744,248,0.899050,0.086209
724,54500,Roger McCreary,CB,18.744735,257,0.748708,0.072937
164,43404,De'Vondre Campbell,ILB,17.045173,241,1.295147,0.070727
603,53447,Jaelan Phillips,OLB,16.169984,233,1.405465,0.069399
147,43338,Jarran Reed,DT,12.213138,205,0.711649,0.059576
298,46142,Jerome Baker,ILB,17.540207,305,0.689669,0.057509


In [9]:
cpr_weighted_blockers['cpyPerPlayDivSTD'] = cpr_weighted_blockers['cpy_per_play'] / cpr_weighted_blockers['std']

In [39]:
## BLOCKER RESULTS

# cpr_weighted_blockers[(cpr_weighted_blockers.counts >= 50) & (cpr_weighted_blockers.position == 'DE')].sort_values('cpy_per_play', ascending=False).shape
# cpr_weighted_no_blockers[cpr_weighted_no_blockers.counts >= 150].sort_values('cpy_per_play', ascending=False).head(20)

# cpr_weighted_blockers[(cpr_weighted_blockers.counts >= 200) & (cpr_weighted_blockers.position == 'CB')].sort_values('cpy_per_play', ascending=False).head(20)
# cpr_weighted_blockers[(cpr_weighted_blockers.counts >= 200) & ((cpr_weighted_blockers.position == 'CB') | (cpr_weighted_blockers.position == 'FS') | (cpr_weighted_blockers.position == 'SS'))].sort_values('cpy_per_play', ascending=False).head(10) #.shape
# cpr_weighted_blockers[(cpr_weighted_blockers.counts >= 150) & (cpr_weighted_blockers.position == 'DT')].sort_values('cpy_per_play', ascending=False).head(20)
# cpr_weighted_blockers[(cpr_weighted_blockers.counts >= 150) & (cpr_weighted_blockers.position == 'DE')].sort_values('cpy_per_play', ascending=False).head(20)
cpr_weighted_blockers[(cpr_weighted_blockers.counts >= 200) & ((cpr_weighted_blockers.position == 'ILB') | (cpr_weighted_blockers.position == 'OLB') | (cpr_weighted_blockers.position == 'MLB'))].sort_values('cpy_per_play', ascending=False).head(20)

,nflId,displayName,position,constrictivePresenceSum,counts,std,cpy_per_play
623,53487,Nick Bolton,ILB,24.799451,258,1.152403,0.096122
624,53489,Pete Werner,OLB,21.379744,248,0.899050,0.086209
164,43404,De'Vondre Campbell,ILB,17.045173,241,1.295147,0.070727
603,53447,Jaelan Phillips,OLB,16.169984,233,1.405465,0.069399
298,46142,Jerome Baker,ILB,17.540207,305,0.689669,0.057509
270,46077,Roquan Smith,ILB,15.027927,286,0.856066,0.052545
602,53445,Zaven Collins,OLB,8.391190,302,1.230230,0.027785
179,43503,Elandon Roberts,ILB,5.004938,215,0.581770,0.023279
275,46085,Tremaine Edmunds,ILB,4.671296,204,1.079602,0.022899
100,42401,Markus Golden,OLB,4.673066,215,0.963866,0.021735


In [135]:
# tackles_by_player_df = tackles_df.groupby(['nflId', 'displayName', 'position']).sum()
# tackles_by_player_df['overallTackles'] = tackles_by_player_df['tackle'] + 0.5 * tackles_by_player_df['assist'] - tackles_by_player_df['pff_missedTackle']
# tackles_by_player_df = tackles_by_player_df.reset_index()
# tackles_by_player_df[tackles_by_player_df['position'] == 'OLB'].sort_values('overallTackles', ascending=False).head(10)

,nflId,displayName,position,gameId,playId,tackle,assist,forcedFumble,pff_missedTackle,overallTackles
337,46304,Zaire Franklin,OLB,190077299706,187055,55,28,0,12,57.0
606,53489,Pete Werner,OLB,163789934473,161197,51,22,2,8,54.0
617,53509,Divine Deablo,OLB,159745724209,158275,39,34,0,7,49.0
278,46088,Leighton Vander Esch,OLB,125370115908,98436,34,25,1,3,43.5
129,42929,Alex Singleton,OLB,141546961172,165625,38,24,0,8,42.0
263,45345,Nicholas Morrow,OLB,137502656771,132206,40,19,0,10,39.5
584,53445,Zaven Collins,OLB,153679537766,147330,39,24,1,13,38.0
70,41300,Christian Kirksey,OLB,121325949277,108520,34,20,0,6,38.0
407,47855,Germaine Pratt,OLB,129414302267,128420,26,31,0,7,34.5
417,47881,Quincy Williams,OLB,115259658573,107325,35,13,0,9,32.5
